In [1]:
!pip install -q nltk
!pip install -q gensim
!pip install -q pyLDAvis
!pip install -q unidecode
!pip install -q matplotlib 
!pip install -q seaborn

In [2]:
!pip install -q langchain==0.2.16 langchain-community==0.2.16 langchain-huggingface==0.0.3 langchain-text-splitters==0.2.4 chromadb==0.5.3 langchain-chroma==0.1.3 pandas ipywidgets

In [15]:
import gensim
import gensim.corpora as corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import pandas as pd
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import pyLDAvis
import pyLDAvis.gensim
import warnings
from gensim.models import CoherenceModel
import html 
import string
from nltk.probability import FreqDist
import matplotlib.pyplot  as plt
from nltk import ngrams
import seaborn as sns
import re 
import unidecode
import pyLDAvis.gensim_models as gensimvis
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from collections import Counter
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
import requests
import json
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.llms import Ollama, BaseLLM
from langchain.schema import Document, Generation, LLMResult
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
from tqdm import tqdm
from glob import glob
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import WordNetLemmatizer
# Download stopwords if not available
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...


True

In [10]:
stop_words = set(stopwords.words('french'))
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# Utils

In [11]:
DICTIONNARY =  ['accord','entreprise', 'preambule', 'sommaire',  'code', 'syndical', 'responsable', 'representant', 
                'present', 'ca', 'organisation', 'preambule', 'peut', 'etre', 'contrat','travail', 'ressources','humaines', 'mise',
                'ainsi', 'et', 'ou', 'alors','collaborateur', 'ci', 'apres', 'party', 'signataire', 'tout', 'etat', 'cause', 'societe', 
                'notamment','article','activite', 'cette', 'donc', 'si', 'sous', 'disposition', 'convention', 'collective', 'dans', 'a', 'cadre',
                'signataire', 'partie', 'parties', 'entre', 'doit'
               ]

DICTIONNARY_STEM = ['part', 'signatair', 'organis', 'syndical', 
                    'dont', 'sieg', 'social', 'conseil', 'prud', 'homm', 
                   'vi', 'professionnel', 'disposit', 'legal', 'conventionnel']

In [16]:
def preprocess_text(text, lang="french"):
    # décoage HTML
    text = html.unescape(text)
    
    # nettoyage de tous les cractères spéciaux
    text = re.sub(r"&[a-z]+;", " ", text)
    text = re.sub(r"&#\d+;", " ", text)
    text = re.sub(r"[<>{}\[\]\|\^\~`\"'=]+", " ", text)
    text = re.sub(r"[–—•«»]+", " ", text)  # Tirets longs, puces, guillemets français

    # tokenisation
    words = word_tokenize(text)

    # lemming
    #stemmer = SnowballStemmer(lang)
                  
    wnl = WordNetLemmatizer()
    words_cleaned = []
    for w in words:
        w_norm = unidecode.unidecode(w.lower())

        if (
            w_norm not in stop_words
            and w_norm not in DICTIONNARY
            and w_norm not in string.punctuation
            and not re.search(r"[<>]|--+|__+|xx+|==+", w_norm)
            and not w_norm.isnumeric()
            and len(w_norm) > 2
        ):
            words_cleaned.append(wnl.lemmatize(w_norm))
            #words_cleaned.append(stemmer.stem(w_norm))

    return words_cleaned


In [17]:
import unidecode
import re

def normalize(text):
    return unidecode.unidecode(text.lower().strip())

def split_text_by_sentences(text, flagged_sentences):
    """
    Découpe le texte en segments basés sur les titres du sommaire, après normalisation.
    """
    split_texts = []
    positions = []

    normalized_text = normalize(text)

    # On garde un mapping (titre original, position) pour préserver les titres initiaux
    for sentence in flagged_sentences:
        norm_sentence = normalize(sentence)
        pos = normalized_text.find(norm_sentence)
        if pos != -1:
            # On retrouve la position réelle dans le texte original
            real_pos = text.lower().find(sentence.lower())
            if real_pos != -1:
                positions.append(real_pos)

    # Si aucune position trouvée, retourner le texte complet
    if not positions:
        return [text]

    positions = sorted(set(positions))
    positions.insert(0, 0)
    positions.append(len(text))

    for i in range(len(positions) - 1):
        start = positions[i]
        end = positions[i + 1]
        split_texts.append(text[start:end].strip())

    return split_texts



In [18]:
text = "Révision-- de l&rsquo;accord : &gt;&gt;' Tous les deux ans, les partenaires sociaux se réunissent. << Suivi de l’accord."
print(preprocess_text(text))


['revision', 'tous', 'deux', 'an', 'partenaires', 'sociaux', 'reunissent', 'suivi']


In [35]:
def split_text_with_titles(text, summary_titles):
    chunks = split_text_by_sentences(text, summary_titles)
    result = {}
    for title in summary_titles:
        for chunk in chunks:
            if normalize(title) in normalize(chunk[:len(title)+30]):
                result[title] = chunk.strip()
                break
    return result


In [19]:
#model_kwargs = {'device': 'cuda'} 
model_kwargs = {'device': 'cpu'}  
MODEL_NAME_EMBEDDER="BAAI/bge-small-en-v1.5"  #petit modèle en anglais
#MODEL_NAME_EMBEDDER="BAAI/bge-m3" #gros modèle multilingue

embedder = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_EMBEDDER, 
    model_kwargs=model_kwargs,
    show_progress=False
)


phrases_non_metier = [
    "Révision de l’accord",
    "Dénonciation de l’accord",
    "Interprétation de l’accord",
    "Suivi de l’accord",
    "Durée de l’accord",
    "Formalités de publicité et de dépôt",
    "Publicité et dépôt",
    "Date d'effet et durée",
    "Champ d'application",
    "Clause de revoyure", 
    "Information des représentants du personnel", 
    "Dispositions relatives à l’accord",
    "Champ d’application",
    "Commission de suivi", 
    "Pause déjeuner du personnel", 
    "Modification de l'accord",
    "Adhésion"
    
]

# Embeddings des phrases non-métier
ref_embeddings = embedder.embed_documents(phrases_non_metier)

def filtre_par_similarite(phrases, seuil=0.85):##torp long utiliser version vectoisée
    results = []
    for phrase in phrases:
        emb = embedder.embed_query(phrase)
        sims = cosine_similarity([emb], ref_embeddings)[0]
        if max(sims) < seuil:
            results.append(phrase) 
    return results

def filtre_par_similarite_vectorise(phrases, seuil=0.85):
    if not phrases:
        return []

    phrase_embeddings = embedder.embed_documents(phrases)  
    sims = cosine_similarity(phrase_embeddings, ref_embeddings)

    # On garde les phrases dont la similarité max avec une phrase non-métier est < seuil
    keep_idx = np.max(sims, axis=1) < seuil
    return [phrase for phrase, keep in zip(phrases, keep_idx) if keep]

    
def filtre_chunks_par_titre(section_dict, phrases_non_metier, seuil=0.85): #seuil arbitraire : en tester plsr
    """
    Ne garde que les chunks dont le titre est peu similaire aux phrases non métier.
    """
    if not section_dict:
        return []

    titres = list(section_dict.keys())
    chunks = list(section_dict.values())

    # Embeddings des titres de section
    titre_embeddings = embedder.embed_documents(titres)
    ref_embeddings = embedder.embed_documents(phrases_non_metier)

    sims = cosine_similarity(titre_embeddings, ref_embeddings)

    # On garde les chunks dont le titre est peu similaire aux phrases non métier
    keep_idx = np.max(sims, axis=1) < seuil
    return [chunk.strip() for chunk, keep in zip(chunks, keep_idx) if keep]



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
phase_1 ="Publicité et dépôt"
phase_2='ARTICLE 7 - PUBLICITE ET DEPOT'

phase_2_embeddings=embedder.embed_documents(phase_2)

sims = cosine_similarity(ref_embeddings, phase_2_embeddings)
print(np.max(sims, axis=1))


[0.63650263 0.66394968 0.66619561 0.61153919 0.63388938 0.66112126
 0.64194487 0.6602614  0.64244291 0.65178487 0.63551024 0.685193
 0.65209934 0.55713831 0.60657047 0.64007168 0.67933923]


In [21]:
filtre_par_similarite_vectorise(["Article 8 – Révision de l’accord", 'Article 5: Contingent annuel d’heures supplémentaires']) 

['Article 5: Contingent annuel d’heures supplémentaires']

In [22]:
#pas parfait 
filtre_par_similarite_vectorise(["Article 8 – Révision de l’accord", 'Article 5: Contingent annuel d’heures supplémentaires', 'Article 4 – Information du Comité Social et Economique', 'Article 5 - Dispositions relatives à l’accord']) 

['Article 5: Contingent annuel d’heures supplémentaires',
 'Article 4 – Information du Comité Social et Economique']

In [23]:
def get_valid_chunks_filtered(section_dict, skip_titles=["préambule", "annexe"], seuil_sim=0.85):
    skip_titles_norm = [normalize(t) for t in skip_titles]

    # supprimer le préambule et avant 
    titles = list(section_dict.keys())
    preamble_idx = next((i for i, t in enumerate(titles) if "préambule" in normalize(t)), -1)
    if preamble_idx != -1:
        titles = titles[preamble_idx + 1:]

    # garder les titres valides uniquement
    valid_titles = [
        t for t in titles if all(skip_kw not in normalize(t) for skip_kw in skip_titles_norm)
    ]
    candidate_dict = {t: section_dict[t] for t in valid_titles}

    # filtrer par similarité des titres
    return filtre_chunks_par_titre(candidate_dict, phrases_non_metier, seuil=seuil_sim)


In [24]:
mots_cles_non_metier = [
    "modification", "publicité", "dépôt", "champ d'application", "durée", 
    "revoyure", "révision", "suivi", "commission", "clause", 
    "formalité", "interprétation", "information"
]

def titre_est_administratif(titre):
    titre_clean = normalize(titre)
    return any(mot in titre_clean for mot in mots_cles_non_metier)

def get_valid_chunks_filtered_bis(section_dict, skip_titles=["préambule", "annexe"], seuil_sim=0.85):
    skip_titles_norm = [normalize(t) for t in skip_titles]

    # Extraire tous les titres
    all_titles = list(section_dict.keys())

    # Chercher l'index du préambule
    preamble_idx = next((i for i, t in enumerate(all_titles) if "préambule" in normalize(t)), -1)

    # Garder seulement les sections à partir du préambule (exclut tout ce qui est avant)
    if preamble_idx != -1:
        filtered_section_dict = {t: section_dict[t] for t in all_titles[preamble_idx:]}
    else:
        filtered_section_dict = section_dict  # Si pas de préambule, on garde tout

    # Garder les titres valides (pas dans les titres à ignorer)
    valid_titles = [
        t for t in filtered_section_dict.keys()
        if all(skip_kw not in normalize(t) for skip_kw in skip_titles_norm)
    ]
    candidate_dict = {t: filtered_section_dict[t] for t in valid_titles}

    if not candidate_dict:
        return []

    # Embeddings des titres
    titres = list(candidate_dict.keys())
    chunks = list(candidate_dict.values())

    titre_embeddings = embedder.embed_documents(titres)
    ref_embeddings = embedder.embed_documents(phrases_non_metier)

    sims = cosine_similarity(titre_embeddings, ref_embeddings)
    keep_sim = np.max(sims, axis=1) < seuil_sim

    # Filtrage combiné : similarité + heuristique
    results = [
        chunk.strip()
        for titre, chunk, keep in zip(titres, chunks, keep_sim)
        if keep and not titre_est_administratif(titre)
    ]
    return results


In [25]:
## exemple d'utilisaton 

dict ={'Préambule :  ': 'Préambule :  \n\nConformément aux dispositions du code du travail, la Direction a invité le CSE, en l’absence d’organisations syndicales représentatives dans la structure  à participer à une négociation collective.\n\nAux termes des réunions en date des 10/10/2022, 16/11/2022 et 12/12/2022 ayant permis de rapprocher les points de vue de chacun, les parties ont abouti à la conclusion du présent accord.',
 'Article 1 – Champ d’application et bénéficiaires\xa0:  ': 'Article 1 – Champ d’application et bénéficiaires\xa0:  \n\nLe présent accord concerne l’ensemble des établissements de l’HADVR.\n\nIl concerne tous les salariés quel que soit leur contrat (CDD ou CDI), quelle que soit leur durée de travail et quel que soit leur métier.\nPar ailleurs, pour répondre aux aspirations des salariés d’une part et aux contraintes inhérentes d’une HAD, les parties se sont accordées pour poursuivre les négociations tout au long de l’année 2023 en vue de la conclusion éventuelle d’un accord sur l’aménagement du temps de travail au sein de la structure.',
 'Article 2\xa0: Rémunération et temps de travail': 'Article 2\xa0: Rémunération et temps de travail',
 '2-1\xa0: Prime de partage de la valeur': "2-1\xa0: Prime de partage de la valeur\n\nDe nombreux investissements matériels et humains ont été réalisés au cours de l’année 2022 pour répondre aux besoins de la structure. Ces investissements auront pour conséquence un budget 2022 non équilibré. \nLe conseil d’administration de la structure, conscient des efforts des professionnels pour poursuivre la montée en charge du nombre de patients accueillis en HAD a répondu favorablement pour le versement d’une prime de partage de la valeur de 300 euros à l’ensemble du personnel excepté la Direction, dans les conditions énoncées ci-après.\n\nAfin de bénéficier des exonérations de cotisation sociales et de l’impôt sur le revenu, est éligible le personnel qui, à la date de versement de la prime, c’est-à-dire au 28 février 2023 :\nLié par un contrat de travail ou d’apprentissage ;\nTravailleurs handicapés liés par un contrat de soutien et d’aide par le travail à un ESAT\xa0;\nLes intérimaires ;\nAyant une rémunération brute inférieure à 3 SMIC conformément aux dispositions légales au cours des 12 mois précédant la date de versement de la prime. \n\nLe salaire annuel brut s’entend de la rémunération annuelle brute (variable et primes inclus) reconstituée en équivalent temps plein sur la période allant de février 2022 à janvier 2023, soit 12 mois. \nIl convient de préciser que la prime versée est calculée au prorata\xa0de la durée de présence effective et du temps de travail contractuel sur la période précitée. \nPar ailleurs et conformément aux dispositions légales, les absences pour congé de maternité, de paternité et d'accueil de l'enfant ou d'adoption, les absences pour congé parental d'éducation, pour enfant malade et pour congé de présence parentale, ainsi que les absences pour accident du travail et maladie professionnelle, sont assimilées à des périodes de présence effective et ne seront donc pas décomptées dans le calcul du temps de travail effectif. \nLa prime sera versée en seule fois avec la paie du mois de février 2023 et figurera sur le bulletin de salaire du mois de versement.\nLa prime ne se substituera à aucun des éléments de rémunération, ni à des augmentations salariales ou prime prévues par un accord, par contrat de travail ou usages en vigueur.",
 '2-2\xa0: Prime «\xa0bas salaires\xa0»\xa0:': '2-2\xa0: Prime «\xa0bas salaires\xa0»\xa0:\n\nLe 28 juin 2022 le Ministre de la transformation et de la fonction publique a annoncé une hausse du point d’indice pour les trois versants de la fonction publique applicable en une fois dès le 1er juillet 2022. Les partenaires sociaux de la branche se sont réunis afin de transposer dans la CCN51 la revalorisation intervenue dans la fonction publique. A l’issue des différentes réunions de négociation qui se sont tenues, aucune organisation syndicale n’a été signataire des textes mis à la signature. La FEHAP a pris une recommandation patronale réévaluant la valeur du point dans la CCN51 en date du 23 novembre 2022. \nDans le contexte inflationniste des derniers mois, compte tenu de la concurrence accrue avec le secteur public, des tensions en matière de recrutement et de la nécessité de fidélisation des professionnels, il est décidé de mettre en en place, par accord d’entreprise, une mesure ciblée pour les «\xa0bas salaires\xa0», en sus de l’augmentation de la valeur du point CCN51.\n\n\nLe conseil d’administration de la structure, conscient que la revalorisation de la valeur du point conventionnel à effet rétroactif au 1er juillet 2022 ne bénéficiera pas au personnel dont le coefficient et donc la rémunération reste à la valeur du SMIC, a répondu favorablement pour le versement d’une prime de 150 euros brute exceptionnelle pour les personnels concernés par ces coefficients au prorata de leur temps de travail contractuel. Cette prime permettra de «\xa0compenser\xa0» la régularisation de la différence sur la valeur du point du 1er juillet au 31 décembre 2022 dont ils ne pourront bénéficier, et sera versée en une fois, en même temps que la régularisation de la valeur du point faite pour les autres membres du personnel sur la paie de janvier 2023.\n\nLes bénéficiaires de la mesure sont tous les professionnels qu’ils soient à temps complet ou à temps partiel, en contrat à durée indéterminée ou en contrat à durée déterminée, qui, au 1er juillet 2022, après application de la valeur du point résultant de la recommandation patronale FEHAP du 23 novembre 2022, sont concernés par l’application de l’article 08-02 de la CCN51 relatif au salaire minimum conventionnel.\n\nCette prime est exclue de l’assiette de calcul de toutes les primes et indemnités prévues par la Convention Collective nationale du 31 octobre 1951.',
 '2-3\xa0: Récupération des heures de fériés et fixation du jour de solidarité pour 2023\xa0:': '2-3\xa0: Récupération des heures de fériés et fixation du jour de solidarité pour 2023\xa0:\n\nLa recommandation patronale du 4 septembre 2012\xa0de la CCN51 avait créé 2 catégories de personnel concernant l’avantage du férié récupéré\xa0: le personnel présent au 1er décembre 2011 ayant pu continuer à bénéficier des anciennes dispositions de la convention du fait d’avantages individuels acquis, et le personnel arrivé après le 1er décembre 2011 qui a dû se voir attribuer les nouveaux critères prévus dans la recommandation patronale. Cela a engendré un souci d’équité.\n\nA compter du 1er janvier 2023, tous les salariés, sans condition d’ancienneté, récupéreront les heures de fériés qu’elles soient travaillées ou non selon les modalités prévues dans la recommandation patronale du 4 septembre 2012.\n\nAu 1er janvier 2023, le don de la journée de solidarité se fera par le biais de la suppression d’une récupération de jour férié (hormis celle due au titre du 1er mai éventuellement générée).\nSi le salarié apporte la preuve (bulletin de salaire faisant mention, attestation, …) qu’il a déjà effectué sous quelque forme que ce soit la journée solidarité au titre de l’année concernée auprès d’un autre employeur, il n’aura pas à l’effectuer au sein de la structure.\nLe salarié ayant plusieurs employeurs effectue sa journée de solidarité chez chacun d’eux au prorata de sa durée contractuelle de travail, de ce fait si le salarié apporte la preuve (bulletin de salaire faisant mention, attestation, …) qu’il a effectué au prorata sa journée ou son don pour la journée solidarité, il ne l’effectuera qu’au prorata au sein de la structure.\nLa journée de solidarité sera évoquée sur le bulletin de salaire de manière à pouvoir apporter la preuve qu’elle a été effectuée dans la structure.\nCas du salarié qui n’a pas pu obtenir de récupération de férié\xa0(pas de férié tombant sur un repos, suspension de contrat ou congé payé durant un férié)\xa0: celui-ci donnera un RTT s’il est concerné par ce dispositif. S’il n’en a pas, il pourra donner un repos conventionnel (tel qu’un repos compensateur de nuit par exemple), sinon il effectuera 7 heures supplémentaires (ou moins selon son temps contractuel) selon les modalités à convenir avec son supérieur hiérarchique de manière à valider son don pour la journée de solidarité.',
 '2-4 : Revalorisation des heures supplémentaires\xa0:': '2-4 : Revalorisation des heures supplémentaires\xa0:\n\nAfin de récompenser les salariés qui accepteraient de remplacer un collègue absent au «\xa0pied levé\xa0», les parties ont convenu de valoriser les heures supplémentaires à hauteur de 150% au lieu de 125% pour toute demande effectuée le vendredi pour le week-end et le lundi, et 24h avant en semaine.',
 '2-5\xa0: Prime parrainage\xa0:': '2-5\xa0: Prime parrainage\xa0:\n\nLa prime de parrainage accordée en 2022 pour toute aide au recrutement de la part des salariés par présentation d’un candidat n’est pas reconduite pour l’année 2023.\nToutefois, une prime de parrainage de 2\xa0500€ brut, est accordée pour toute aide au recrutement d’un médecin praticien d’HAD (0,80 à 1 ETP) et versée à la fin de la période d’essai du professionnel.',
 'Article 3\xa0: conditions de travail': 'Article 3\xa0: conditions de travail\n\n3-1\xa0: Casiers nominatifs sur chaque antenne\xa0:\n\nDe nouvelles antennes et locaux ont été aménagés en 2022. Pour répondre à la problématique d’accueil de nouveaux collaborateurs et le travail en mobilité sur plusieurs antennes, les parties se sont accordées sur l’agencement de bureaux partagés nécessitant la mise à disposition de casiers nominatifs au sein de chaque antenne.\nLa direction s’engage à réaliser les achats nécessaires pour la mise à disposition de ces casiers nominatifs au sein de chaque antenne dès l’agencement terminé.',
 '3-2\xa0: pause déjeuner du personnel\xa0:': '3-2\xa0: pause déjeuner du personnel\xa0:\n\nLa demande des salariés est de réduire le temps de présence journalier sur leur lieu de travail et de diminuer le temps accordé à la pause repas à 30 mn au lieu d’une heure.\n\nLes parties s’accordent sur une pause de 30 mn à condition que cela n’affecte pas le fonctionnement du service. Les horaires de travail seront ajustés par les responsables en fonction de l’amplitude de la pause repas et devront correspondre aux besoins de l’établissement.',
 '3-3\xa0: utilisation voitures de service\xa0:': '3-3\xa0: utilisation voitures de service\xa0:\n\nLe personnel soignant pourra garder le véhicule de service en cas de travail sur 2 jours consécutifs, par nécessité de service. En contrepartie, le salarié s’engage, par tout moyen, à restituer le véhicule de service en cas d’absence non programmée. Cf modalités dans le règlement intérieur des véhicules de service signé par le personnel avec attestation de remisage.',
 'Article 4 – Information du Comité Social et Economique': 'Article 4 – Information du Comité Social et Economique\nLe CSE sera informé du présent accord lors de réunion du 19 janvier 2023, dans le cadre de sa mission au titre de l’article L2312-8 du code du travail.',
 'Article 5 - Dispositions relatives à l’accord ': 'Article 5 - Dispositions relatives à l’accord \nLe présent accord entre en application après son dépôt sur la plateforme de téléprocédure en application des conditions légales et réglementaires en vigueur, pour une durée indéterminée.\nLe présent accord est également déposé au secrétariat-greffe du Conseil des Prud’hommes de Libourne.\nIl pourra être révisé conformément aux dispositions légales.\nIl fait l’objet des mesures de publicité prévues par les dispositions légales et réglementaires sur les lieux d’affichage habituels.\n\nFait à Libourne, le 19 janvier 2023, \n\n\nSignature de la Direction\xa0:\n\n\nSignatures des membres titulaires du CSE\xa0:'}

In [26]:
print(get_valid_chunks_filtered_bis(dict, skip_titles=["préambule", "annexe"], seuil_sim=0.85))

['Article 2\xa0: Rémunération et temps de travail', "2-1\xa0: Prime de partage de la valeur\n\nDe nombreux investissements matériels et humains ont été réalisés au cours de l’année 2022 pour répondre aux besoins de la structure. Ces investissements auront pour conséquence un budget 2022 non équilibré. \nLe conseil d’administration de la structure, conscient des efforts des professionnels pour poursuivre la montée en charge du nombre de patients accueillis en HAD a répondu favorablement pour le versement d’une prime de partage de la valeur de 300 euros à l’ensemble du personnel excepté la Direction, dans les conditions énoncées ci-après.\n\nAfin de bénéficier des exonérations de cotisation sociales et de l’impôt sur le revenu, est éligible le personnel qui, à la date de versement de la prime, c’est-à-dire au 28 février 2023 :\nLié par un contrat de travail ou d’apprentissage ;\nTravailleurs handicapés liés par un contrat de soutien et d’aide par le travail à un ESAT\xa0;\nLes intérimair

In [27]:
print(get_valid_chunks_filtered(dict, skip_titles=["préambule", "annexe"], seuil_sim=0.85))

['Article 1 – Champ d’application et bénéficiaires\xa0:  \n\nLe présent accord concerne l’ensemble des établissements de l’HADVR.\n\nIl concerne tous les salariés quel que soit leur contrat (CDD ou CDI), quelle que soit leur durée de travail et quel que soit leur métier.\nPar ailleurs, pour répondre aux aspirations des salariés d’une part et aux contraintes inhérentes d’une HAD, les parties se sont accordées pour poursuivre les négociations tout au long de l’année 2023 en vue de la conclusion éventuelle d’un accord sur l’aménagement du temps de travail au sein de la structure.', 'Article 2\xa0: Rémunération et temps de travail', "2-1\xa0: Prime de partage de la valeur\n\nDe nombreux investissements matériels et humains ont été réalisés au cours de l’année 2022 pour répondre aux besoins de la structure. Ces investissements auront pour conséquence un budget 2022 non équilibré. \nLe conseil d’administration de la structure, conscient des efforts des professionnels pour poursuivre la monté

In [28]:
def lda_model(processed_texts, num_topics=5, passes=10):
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    warnings.filterwarnings('ignore')
    dictionary = corpora.Dictionary(processed_texts)
    corpus = [dictionary.doc2bow(text) for text in processed_texts]
    model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, passes=passes)
    for topic in model.print_topics(num_words=5):
        print(topic)
    return model, corpus, dictionary
    
def visualize_lda(model, corpus, dictionary):
    pyLDAvis.enable_notebook()
    vis_data = gensimvis.prepare(model, corpus, dictionary)
    return pyLDAvis.display(vis_data)

In [29]:
def compute_coherence_values(dictionary, corpus, texts, max_topics=10):
    coherence_scores = []
    for num_topics in range(2, max_topics + 1):
        lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=10, random_state=42)
        coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append((num_topics, coherence_score))
        print(f"Num Topics: {num_topics}, Coherence Score: {coherence_score:.4f}")
    
    return coherence_scores

# Pour HS

In [30]:
sommaire_hs = pd.read_parquet("data/echantillon_1000_hs_accords_TOC.parquet")
df_hs = pd.read_parquet("data/echantillon_1000_hs_accords.parquet")
#sommaire_hs

In [33]:
df_hs["summary"] = sommaire_hs["extracted_summary"]

df_hs["summary"][50]

array(['ACCORD D’ENTREPRISE RELATIF',
       'AU CONTINGENT ANNUEL D’HEURES SUPPLEMENTAIRES',
       'ET A LA DUREE MAXIMALE JOURNALIERE DE TRAVAIL', 'PREAMBULE',
       'Article 1\xa0: Objet', 'Article 2\xa0: Champ d’application',
       'Article\xa03\xa0: Définition du temps de travail effectif',
       'Article 4\xa0: Accomplissement d’heures supplémentaires',
       'Article 5\xa0: Contingent annuel d’heures supplémentaires',
       'Article 6\xa0: Durée maximale journalière de travail',
       'Article 7\xa0: Durée de l’accord',
       'Article 8 – Révision de l’accord',
       'Article 9 – Dénonciation de l’accord',
       'Article 10 – Interprétation de l’accord ',
       'Article 11 – Suivi de l’accord',
       'Article 12\xa0: Formalités de publicité et de dépôt'],
      dtype=object)

In [36]:
df_hs["section_dict"] = df_hs.apply(
    lambda row: split_text_with_titles(row["accorddocx"], row["summary"]),
    axis=1
)

#df_hs["section_dict"][0]

In [37]:
# Application dans le DataFrame
df_hs["lda_documents"] = df_hs["section_dict"].apply(get_valid_chunks_filtered)
# Nettoyage NLP + lemming
all_chunks_hs = [chunk for doc_chunks in df_hs["lda_documents"] for chunk in doc_chunks]
processed_texts_hs = [preprocess_text(doc) for doc in all_chunks_hs]

In [ ]:
#df_hs["lda_documents"][0]
# toujours "Article 4 – Information du Comité Social et Economique" mais le reste a été filtré

In [38]:
model_hs, corpus_hs, dictionary_hs = lda_model(processed_texts_hs)

(0, '0.022*"salarie" + 0.012*"salary" + 0.011*"delai" + 0.010*"ca" + 0.010*"jours"')
(1, '0.031*"temp" + 0.021*"salary" + 0.009*"service" + 0.009*"prime" + 0.008*"nuit"')
(2, '0.108*"heures" + 0.037*"duree" + 0.033*"supplementaires" + 0.031*"temp" + 0.022*"repos"')
(3, '0.063*"jours" + 0.029*"conge" + 0.025*"periode" + 0.024*"nombre" + 0.024*"salarie"')
(4, '0.013*"depot" + 0.012*"personnel" + 0.012*"part" + 0.011*"social" + 0.010*"duree"')


In [39]:
visualize_lda(model_hs, corpus_hs, dictionary_hs)

In [ ]:
compute_coherence_values(dictionary_hs, corpus_hs, processed_texts_hs, max_topics=20)

Num Topics: 2, Coherence Score: 0.4018
Num Topics: 3, Coherence Score: 0.4909


In [45]:
model_hs, corpus_hs, dictionary_hs = lda_model(processed_texts_hs, num_topics= 18)

(0, '0.029*"duree" + 0.026*"revision" + 0.022*"vigueur" + 0.020*"mois" + 0.020*"disposition"')
(1, '0.046*"prime" + 0.023*"EUR" + 0.019*"indemnite" + 0.015*"montant" + 0.011*"salary"')
(2, '0.088*"temp" + 0.063*"repos" + 0.038*"salarie" + 0.032*"effectif" + 0.018*"pause"')
(3, '0.017*"droit" + 0.017*"salarie" + 0.016*"salary" + 0.015*"teletravail" + 0.014*"deconnexion"')
(4, '0.044*"part" + 0.037*"social" + 0.025*"representee" + 0.024*"qualite" + 0.024*"dont"')
(5, '0.100*"periode" + 0.061*"reference" + 0.046*"cours" + 0.043*"annee" + 0.028*"salarie"')
(6, '0.087*"temp" + 0.063*"salary" + 0.026*"partiel" + 0.020*"duree" + 0.017*"cadre"')
(7, '0.114*"jours" + 0.051*"conge" + 0.034*"nombre" + 0.029*"paye" + 0.029*"annee"')
(8, '0.046*"jours" + 0.045*"salarie" + 0.035*"forfait" + 0.024*"charge" + 0.022*"repos"')
(9, '0.036*"heures" + 0.031*"service" + 0.028*"horaires" + 0.021*"horaire" + 0.017*"plage"')
(10, '0.064*"conge" + 0.044*"absence" + 0.038*"anciennete" + 0.033*"salarie" + 0.033*"

In [46]:
visualize_lda(model_hs, corpus_hs, dictionary_hs)

In [47]:
# Application dans le DataFrame
df_hs["lda_documents_bis"] = df_hs["section_dict"].apply(get_valid_chunks_filtered_bis)
# Nettoyage NLP + lemming
all_chunks_hs_bis = [chunk for doc_chunks in df_hs["lda_documents_bis"] for chunk in doc_chunks]
processed_texts_hs_bis = [preprocess_text(doc) for doc in all_chunks_hs_bis]


In [48]:
model_hs_bis, corpus_hs_bis, dictionary_hs_bis = lda_model(processed_texts_hs_bis)


(0, '0.018*"part" + 0.016*"personnel" + 0.014*"social" + 0.012*"dont" + 0.011*"representee"')
(1, '0.055*"jours" + 0.037*"temp" + 0.025*"salarie" + 0.023*"periode" + 0.021*"nombre"')
(2, '0.017*"conge" + 0.016*"salarie" + 0.012*"jours" + 0.011*"salary" + 0.009*"service"')
(3, '0.020*"salary" + 0.017*"temp" + 0.012*"disposition" + 0.011*"duree" + 0.007*"salarie"')
(4, '0.134*"heures" + 0.043*"supplementaires" + 0.037*"duree" + 0.023*"repos" + 0.018*"hebdomadaire"')


In [49]:
visualize_lda(model_hs_bis, corpus_hs_bis, dictionary_hs_bis)

# Pour les données de santé

In [50]:
df_sante= pd.read_parquet("data/complementaire_sante_580.parquet")
df_sante

,extracted_summary,accorddocx,num_dossier
0,"[Garanties Collectives frais de santé, indemni...",\n\n\n\n\nAVENANT N°1\n\nA L’ACCORD COLLECTIF ...,T08421002987
1,[Chapitre I – Champ d’application et mise en œ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\nFDSEA MAINE ET LOI...,T04919002866
2,"[ACCORD D’ENTREPRISE INSTITUANT, UN REGIME DE...",\n\nACCORD D’ENTREPRISE INSTITUANT\n UN REGIME...,T01822001378
3,"[ACCORD COMPLEMENTAIRE SANTE - NON - CADRES, G...",\nACCORD COMPLEMENTAIRE SANTE - NON - CADRES\n...,T04421012705
4,[AVENANT N°5 A L’ACCORD D’ENTREPRISE RELATIF A...,\nAVENANT N°5 A L’ACCORD D’ENTREPRISE RELATIF ...,T07522049321
...,...,...,...
575,[ACCORD D’ENTREPRISE RELATIF A LA MODIFICATION...,\nACCORD D’ENTREPRISE RELATIF A LA MODIFICATIO...,T06720004275
576,"[Préambule, CHAPITRE 1 : STATUT COLLECTIF, Art...",\n\n \n\n\nACCO...,T03520005800
577,[AVENANT N°3 A L’ACCORD ASSURANCE COMPLEMENTAI...,AVENANT N°3 A L’ACCORD ASSURANCE COMPLEMENTAIR...,T00323002687
578,[TITRE I - DISPOSITIONS GENERALES REGISSANT LE...,ACCORD ANTICIPE D’ADAPTATION\nRELATIF AU STATU...,T07519010842


In [51]:
df_sante["extracted_summary"][0]

array(['Garanties Collectives frais de santé, indemnisations complémentaires aux prestations versées par le régime de base de la Sécurité Sociale.',
       'PREAMBULE', 'ARTICLE 1 – OBJET DE L’AVENANT N°1',
       'ARTICLE 2 – MODIFICATION DE LA PART PATRONALE',
       'ARTICLE 3 – DATE D’EFFET',
       'ARTICLE 4 – DUREE, DENONCIATION, REVISION DE L’AVENANT N°1',
       'ARTICLE 5 - ADHESION', 'ARTICLE 6 – MODIFICATION DE L’AVENANT',
       'ARTICLE 7 - PUBLICITE ET DEPOT'], dtype=object)

In [52]:
df_sante["section_dict"] = df_sante.apply(
    lambda row: split_text_with_titles(row["accorddocx"], row["extracted_summary"]),
    axis=1
)


In [ ]:
#df_sante["section_dict"][5]

In [53]:
# Application dans le DataFrame
df_sante["lda_documents"] = df_sante["section_dict"].apply(get_valid_chunks_filtered)

# Nettoyage NLP + lemming
all_chunks_sante = [chunk for doc_chunks in df_sante["lda_documents"] for chunk in doc_chunks]
processed_texts_sante = [preprocess_text(doc) for doc in all_chunks_sante]


In [ ]:
#df_sante["lda_documents"][0]

In [ ]:
model_sante, corpus_sante, dictionary_sante = lda_model(processed_texts_sante)

In [ ]:
visualize_lda(model_sante, corpus_sante, dictionary_sante)

In [ ]:
compute_coherence_values(dictionary_sante, corpus_sante, processed_texts_sante, max_topics=20)

In [54]:
model_sante, corpus_sante, dictionary_sante = lda_model(processed_texts_sante, num_topics= 17)

(0, '0.089*"conge" + 0.050*"jours" + 0.023*"paye" + 0.020*"salarie" + 0.018*"annee"')
(1, '0.041*"salary" + 0.028*"regime" + 0.021*"adhesion" + 0.020*"couverture" + 0.017*"obligatoire"')
(2, '0.030*"disposition" + 0.026*"revision" + 0.026*"duree" + 0.022*"signataires" + 0.021*"vigueur"')
(3, '0.019*"part" + 0.017*"representee" + 0.016*"syndicale" + 0.015*"ete" + 0.015*"negociation"')
(4, '0.037*"jours" + 0.033*"heures" + 0.028*"temp" + 0.021*"salarie" + 0.016*"repos"')
(5, '0.057*"information" + 0.026*"comite" + 0.024*"notice" + 0.022*"garanties" + 0.022*"economique"')
(6, '0.022*"salarie" + 0.020*"garanties" + 0.018*"charge" + 0.018*"maintien" + 0.018*"salary"')
(7, '0.035*"salary" + 0.031*"vandemoortele" + 0.024*"europe" + 0.022*"france" + 0.021*"etablissement"')
(8, '0.033*"retraite" + 0.031*"chapitre" + 0.029*"cet" + 0.025*"disposition" + 0.023*"compte"')
(9, '0.133*"prime" + 0.044*"EUR" + 0.034*"montant" + 0.021*"euro" + 0.021*"mois"')
(10, '0.034*"regime" + 0.028*"sante" + 0.023*

In [55]:
visualize_lda(model_sante, corpus_sante, dictionary_sante)

In [56]:
# Application dans le DataFrame
df_sante["lda_documents_bis"] = df_sante["section_dict"].apply(get_valid_chunks_filtered_bis)
# Nettoyage NLP + lemming
all_chunks_sante_bis = [chunk for doc_chunks in df_sante["lda_documents_bis"] for chunk in doc_chunks]
processed_texts_sante_bis = [preprocess_text(doc) for doc in all_chunks_sante_bis]

In [57]:
model_sante_bis, corpus_sante_bis, dictionary_sante_bis = lda_model(processed_texts_sante_bis)

(0, '0.015*"sante" + 0.012*"disposition" + 0.010*"complementaire" + 0.010*"avenant" + 0.010*"regime"')
(1, '0.026*"jours" + 0.021*"temp" + 0.018*"conge" + 0.017*"heures" + 0.012*"salarie"')
(2, '0.012*"professionnelle" + 0.009*"formation" + 0.008*"salary" + 0.008*"personnel" + 0.007*"emploi"')
(3, '0.023*"salary" + 0.022*"salarie" + 0.018*"regime" + 0.013*"ca" + 0.012*"employeur"')
(4, '0.019*"prime" + 0.010*"augmentation" + 0.010*"salary" + 0.009*"direction" + 0.008*"1er"')


In [58]:
visualize_lda(model_sante_bis, corpus_sante_bis, dictionary_sante_bis)